# データ解析サンプル1

## 1. データソースについて

「令和５年における労働災害発生状況（確定）(23-16.xlsx）をもとに、データ解析を行う。

ここでは、同Excelファイルの「都道府県、業種別死亡災害発生状況（令和５年）」のデータを対象する。

## 2. 前処理

### 前処理の必要性

同Excelファイルは複数のシートに異なる対象のデータが保存され、さらに読者がそのまま読む上で分かりやすく工夫がされている。代表的なものを以下に示す。

* 表紙ページがある
* タイトル文字列や、データを取得した日など、表そのものの齟齬側の情報が記載されている
* １つのシートに２つの表がある
* 表の中に小計がある

これらの表データ以外の情報は自動で認識することができないため、あらかじめ処理しやすい形式に自動、手動で変換しておく必要がある。これを前処理と呼ぶ。

### どのような前処理をしたか

前処理の内容を以下に示す。

1. 元のExcelファイルのシート「都道府県、業種別死亡災害発生状況（令和５年）」を取り出す
2. 不要な行、列を削除
3. 小計、合計の欄を削除
4. 小分類（製造、鉱業、建設、運輸、貨物、農林、他）ごとにシートを分割

## 3. Excelファイルの読み込み


In [ ]:
import numpy as np
import pandas as pd

EXCEL_FILE = 'data/都道府県、業種別死亡災害発生状況（令和５年）.xlsx'
dfs = pd.read_excel(EXCEL_FILE, sheet_name=None, index_col=0)
dfs.keys()

## 4.製造分野のDataFrameを取り出し

In [ ]:
df_prod = dfs['製造']
df_prod

## 5. 県別の製造業死亡災害発生数を表示

* `df.sum()`で合計値を計算
* `axis=1`でカラム集計を指定（0ならインデックスで集計）

In [ ]:
df_prod.sum(axis=1)

## 6. ソートして上位と下位を表示

* ソートは`df.sort_values()`を使う
  * デフォルトで昇順（少ない方から多い方へ）
  * `ascending=False`で降順
* 先頭の取り出しは`df.head()`
* 末尾の取り出しは`df.tail()`

In [ ]:
# 県別の製造業死亡災害発生数のトップ5
df_prod.sum(axis=1).sort_values(ascending=False).head(5)

In [ ]:
# 県別の製造業死亡災害発生数のトップ5
df_prod.sum(axis=1).sort_values(ascending=False).tail(5)

## 7. カラム（小分類）ごとの集計

`df.sum(axis=0)`で集計する。

In [ ]:
# 小分類ごとの製造業死亡災害発生数
df_prod.sum(axis=0)

In [ ]:
# さらにソートしてトップ5
df_prod.sum(axis=0).sort_values(ascending=False).head(5)

## 8. 全体を集計する

DataFrame全体（製造業の死亡災害発生数）の合計値を出す。

In [ ]:
# 製造業の死亡災害発生数
print(df_prod.sum().sum())   # axis=0は省略
print(df_prod.values.sum())  # こういう書き方もある

全シート（全分類）の合計値を出す。

In [ ]:
# 死亡災害発生数の合計
total = np.int64(0)  # 単に0を代入してもいい
for key, df in dfs.items():
    sub_total = df.sum().sum()
    print(key, sub_total)  # 分類ごとの小計を表示
    total += sub_total

print('合計', total)